## Notebook to Calculate Game Goal Differntial from Kaggle and EH data
#### 2007/2008-2019/2020 are from Kaggle, 2021- is from EH via Kevin

### 2007-2019
- open the team_game_stats csv and group by game_id
- for each game_id, get the home team, away team, home score, away score

### 2020-2022
- for each season, we have a csv with all the goals. 
- open the file, group it by game_id, and use groupby to get the sum of home/away goals that were not 1v0 or 0v1 gamestate (not shootout)
- add the game_id, home team, away team, home score, away score to a csv

In [1]:
import pandas as pd

In [2]:
pre_19_game_df = pd.read_csv('/Users/tylerviducic/dev/hockey_analytics/gamescore_model/kaggle_data/game.csv')
print(pre_19_game_df.head().to_markdown())

|    |    game_id |   season | type   | date_time_GMT        |   away_team_id |   home_team_id |   away_goals |   home_goals | outcome      | home_rink_side_start   | venue          | venue_link          | venue_time_zone_id   |   venue_time_zone_offset | venue_time_zone_tz   |
|---:|-----------:|---------:|:-------|:---------------------|---------------:|---------------:|-------------:|-------------:|:-------------|:-----------------------|:---------------|:--------------------|:---------------------|-------------------------:|:---------------------|
|  0 | 2016020045 | 20162017 | R      | 2016-10-19T00:30:00Z |              4 |             16 |            4 |            7 | home win REG | right                  | United Center  | /api/v1/venues/null | America/Chicago      |                       -5 | CDT                  |
|  1 | 2017020812 | 20172018 | R      | 2018-02-07T00:00:00Z |             24 |              7 |            4 |            3 | away win OT  | left                 

In [3]:
pre_19_game_team_stats_df = pd.read_csv('/Users/tylerviducic/dev/hockey_analytics/gamescore_model/kaggle_data/game_teams_stats.csv')
print(pre_19_game_team_stats_df.head().to_markdown())

|    |    game_id |   team_id | HoA   | won   | settled_in   | head_coach       |   goals |   shots |   hits |   pim |   powerPlayOpportunities |   powerPlayGoals |   faceOffWinPercentage |   giveaways |   takeaways |   blocked | startRinkSide   |
|---:|-----------:|----------:|:------|:------|:-------------|:-----------------|--------:|--------:|-------:|------:|-------------------------:|-----------------:|-----------------------:|------------:|------------:|----------:|:----------------|
|  0 | 2016020045 |         4 | away  | False | REG          | Dave Hakstol     |       4 |      27 |     30 |     6 |                        4 |                2 |                   50.9 |          12 |           9 |        11 | left            |
|  1 | 2016020045 |        16 | home  | True  | REG          | Joel Quenneville |       7 |      28 |     20 |     8 |                        3 |                2 |                   49.1 |          16 |           8 |         9 | left            |
|  2 | 2

In [4]:
team_stats_grouped = pre_19_game_team_stats_df.groupby('game_id')

In [13]:
away_df = pd.concat([group.loc[group['HoA'] == 'away', ['game_id', 'team_id', 'goals']].reset_index(drop=True) for game, group in team_stats_grouped])
away_df = away_df.rename(columns={'team_id': 'away_team_id', 'goals': 'away_goals'})
home_df = pd.concat([group.loc[group['HoA'] == 'home', ['game_id', 'team_id', 'goals']].reset_index(drop=True) for game, group in team_stats_grouped])
home_df = home_df.rename(columns={'team_id': 'home_team_id', 'goals': 'home_goals'})
goal_diff_df = pd.merge(away_df, home_df, on='game_id')
goal_diff_df['home_goals'] = goal_diff_df['home_goals'].astype("Int64") # Int64 allows for null values
goal_diff_df['away_goals'] = goal_diff_df['away_goals'].astype("Int64")


In [15]:
# TODO: follow the procedure for the 2020-2022 seasons